In [55]:
import time
import sqlite3
import requests
import datetime

import pandas as pd
import shioaji as sj
import re
from shioaji import TickFOPv1, Exchange
from exchange_calendars import get_calendar

import mplfinance as mpf

from FuncBase import *

In [56]:
import os 
import shioaji as sj
from dotenv import load_dotenv

#永豐金API登入
load_dotenv()
api = sj.Shioaji()
api.login(os.getenv('API_KEY'),os.getenv('SECRET_KEY'),  
          contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

#連結DB Browser
connection = sqlite3.connect('work_data.db')

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


<SecurityType.Index: 'IND'> fetch done
<SecurityType.Future: 'FUT'> fetch done
<SecurityType.Option: 'OPT'> fetch done
<SecurityType.Stock: 'STK'> fetch done


In [76]:
import time
def get_ticks(connection, api, dates, codes = str, is_Futures = False):

    tw_calendar = get_calendar('XTAI')

    #name tickers and correct time stamp

    if is_Futures:
        
        #若為期貨只取英文代號作為code name
        code = ''.join(char for char in codes if char.isalpha())

        if dates.hour > 15:
            date = dates+datetime.timedelta(days=1)
        
        else:
            date = dates

    else:
        code = codes
        date = dates
        

    try:    
        sql = "SELECT * FROM ticks WHERE code = '{}' and ts BETWEEN '{}' AND '{}' ".format(code,
                                                                                 date,
                                                                                 date+datetime.timedelta(days=1))
        df = pd.read_sql(sql, connection, parse_dates = ['ts'], index_col=['ts'])
    except:
        df = pd.DataFrame()
    
    if not df.empty:
        return df, True
    
    if is_Futures: #若為期貨
            ticks = api.ticks(
                contract = api.Contracts.Futures.get(code)[codes],  # For futures, use the Futures contract
                date=date.strftime('%Y-%m-%d') 
                )

    else: #若為證券
            ticks = api.ticks(
            contract=api.Contracts.Stocks[code],  # For stocks, use the Stocks contract
                date=date.strftime('%Y-%m-%d')
            )    
    df = pd.DataFrame({**ticks})

    df.ts = pd.to_datetime(df.ts)
    df['code'] = code
    df = df.set_index('ts')

    return df, False

In [77]:
dates = pd.to_datetime('2024-10-28')
df, in_db = get_ticks(connection, api, dates, '2330')


print(in_db)
df

False


,tick_type,bid_price,bid_volume,ask_volume,volume,close,ask_price,code
ts,,,,,,,,
2024-10-28 09:00:02.949493,1,1075.0,899,2632,2053,1075.0,1080.0,2330
2024-10-28 09:00:03.032777,1,1075.0,899,2631,1,1080.0,1080.0,2330
2024-10-28 09:00:03.253842,2,1075.0,1054,2682,1,1075.0,1080.0,2330
2024-10-28 09:00:05.759067,1,1075.0,1132,2694,1,1080.0,1080.0,2330
2024-10-28 09:00:08.038434,2,1075.0,1434,2725,3,1075.0,1080.0,2330
...,...,...,...,...,...,...,...,...
2024-10-28 13:24:59.449575,1,1055.0,1066,188,1,1060.0,1060.0,2330
2024-10-28 13:24:59.696658,1,1055.0,1066,185,3,1060.0,1060.0,2330
2024-10-28 13:24:59.700689,1,1055.0,1066,182,3,1060.0,1060.0,2330


In [60]:
def update_ticks(connection, api, daily_target, is_Futures = False):
    main_df = pd.DataFrame()

    tw_calendar = get_calendar('XTAI')

    for date, codes in daily_target.items():

        day_trading_codes = [code for code in codes ]

        print('正在更新{}逐筆成交資料，總共{}檔標的，為{}'.format(date.strftime('%Y/%m/%d'), len(day_trading_codes),day_trading_codes))

        for code in day_trading_codes:
            df, in_db = get_ticks(connection, api, date, code, is_Futures)

            if df is not None and in_db:
                main_df = pd.concat([main_df, df], sort = False)
                time.sleep(1)

            prev_trading_date = tw_calendar.previous_close(date).date()
            prev_df, prev_in_db = get_ticks(connection, api, date, code, is_Futures)

            if prev_df is not None and not prev_in_db:
                main_df = pd.concat([main_df, prev_df], sort = False)
                time.sleep(1)
    
    if not main_df.empty:
        try:
            main_df.to_sql('ticks', connection, if_exists='append', index=False)
            print("Data stored successfully.")
        except Exception as e:
            print("Failed to store data:", e)
    return main_df



In [61]:
daily_targets ={
    pd.to_datetime('2024-08-24'): ['2317', '2330'],
    pd.to_datetime('2024-08-25'): ['2317', '2330'],
    pd.to_datetime('2024-08-28'): ['2317', '2330']
                }

In [62]:
connection = sqlite3.connect('work_data.db')
update_ticks(connection, api, daily_targets)

正在更新2024/08/24逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/08/25逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/08/28逐筆成交資料，總共2檔標的，為['2317', '2330']
Data stored successfully.


,tick_type,bid_price,bid_volume,ask_volume,volume,close,ask_price,code
ts,,,,,,,,
2024-08-28 09:00:01.330251,1.0,181.0,168.0,369.0,1193.0,181.0,181.5,2317
2024-08-28 09:00:01.369385,2.0,181.0,146.0,369.0,22.0,181.0,181.5,2317
2024-08-28 09:00:01.369417,2.0,181.0,124.0,369.0,22.0,181.0,181.5,2317
2024-08-28 09:00:01.370082,2.0,181.0,84.0,369.0,40.0,181.0,181.5,2317
2024-08-28 09:00:01.438087,1.0,181.0,85.0,369.0,1.0,181.5,181.5,2317
...,...,...,...,...,...,...,...,...
2024-08-28 13:24:57.732447,1.0,962.0,170.0,412.0,2.0,963.0,963.0,2330
2024-08-28 13:24:58.418472,2.0,962.0,170.0,412.0,1.0,962.0,963.0,2330
2024-08-28 13:24:58.492563,2.0,962.0,169.0,412.0,1.0,962.0,963.0,2330


In [63]:
def retry_request(url, payloads, headers):
    
    for i in range(3):
    
        try:
            return requests.get(url, params= payloads, headers=headers)
        
        except:
            print('發生錯誤，請等待一分種後再嘗試')
            time.sleep(60)
    
    return None

def get_daily_prices(date, connection):

    try:
        df = pd.read_sql("select * from daily_prices where 日期 = '{}'".format(date),
                         connection,
                         parse_dates= ['日期'],
                         index_col= ['證券代號','日期'])
    except:
        df = pd.DataFrame()
    
    if not df.empty:
        return df, True
    #INDB TRUE
    #先在資料庫裡抓取每日收盤價，沒有資料則利用爬蟲重新抓取
    
    url = 'https://www.twse.com.tw/exchangeReport/MI_INDEX'

    payloads = {
        'response': 'html',
        'date': date.strftime('%Y%m%d'),
        'type': 'ALLBUT0999'
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.80 Safari/537.36'
    }

    response = retry_request(url, payloads, headers)

    try:
        df = pd.read_html(response.text)[-1]
    
    except:
        print('{} 找不到資料'.format(
            date.strftime('%Y%m%d')))
        
        return None, False
    #INDB False
    df.columns = df.columns.get_level_values(2)
    df['漲跌價差'] = df['漲跌價差'].where(df['漲跌(+/-)'] != '-', -df['漲跌價差'])
    df.drop(['證券名稱','漲跌(+/-)'],inplace=True, axis=1)
    df['日期'] = pd.to_datetime(date)
    df = df.set_index(['證券代號','日期'])
    df = df.apply(pd.to_numeric, errors = 'coerce')
    df.drop(df[df['收盤價'].isnull()].index, inplace=True)
    df['昨日收盤價'] = df['收盤價'] - df['漲跌價差']
    df['股價震幅'] = (df['最高價'] - df['最低價']) / df['昨日收盤價']*100

    return df, False



In [64]:
def update_daily_prices(start_date, end_date, connection):

    tw_calendars = get_calendar("XTAI")

    main_df = pd.DataFrame()

    for date in pd.date_range(start_date, end_date):

        if date not in tw_calendars.opens:
            continue

        df, in_db = get_daily_prices(date, connection)

        if df is not None and not in_db:
            main_df = pd.concat([main_df,df],axis = 0)
            print('{} 每日收盤行情抓取完成，等待15秒'.format(date.strftime('%Y%m%d')))
            time.sleep(15)


    if not main_df.empty:
        main_df.to_sql('daily_price', connection, if_exists='append')
        
        return main_df

    

In [65]:
def update_historical_data(start_date, end_date, daily_target, connection, api):

    tw_calendar = get_calendar('XTAI')

    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)

    print('正在更新每日收盤行情...')

    update_daily_prices(tw_calendar.previous_close(start).date(), end, connection)

    print('正在更新逐筆交易...')
    
    update_ticks(connection, api, daily_targets)

    print('股市資料更新完成')


In [67]:
update_historical_data('2024-10-26','2024-10-28', '2330', connection, api)

正在更新每日收盤行情...
20241025 每日收盤行情抓取完成，等待15秒
20241028 每日收盤行情抓取完成，等待15秒
正在更新逐筆交易...
正在更新2024/08/24逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/08/25逐筆成交資料，總共2檔標的，為['2317', '2330']
正在更新2024/08/28逐筆成交資料，總共2檔標的，為['2317', '2330']
Data stored successfully.
股市資料更新完成


In [68]:
ticks = {}
kbars = {}
volume_today = {}

In [82]:
def tbot(codes, is_Futures = False):
    import pandas as pd
    
    print('--- 啟動 Tbot #{} ---'.format(codes))

    tw_calendar = get_calendar('XTAI')
    
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return
    
    if is_Futures:
        code = ''.join(char for char in codes if char.isalpha())
    else:
        code = codes
    
    ticks[code] = pd.concat([get_ticks(connection, api, prev_trading_date, codes, is_Futures)[0],get_ticks(connection, api, date, codes, is_Futures)[0]])
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()
    
    if is_Futures:
        api.quote.subscribe(api.Contracts.Futures.get(codes[0:3])[codes], 
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1 
                            )
        

    else:
        api.quote.subscribe(api.Contracts.Stocks[codes],
                            quote_type= 'tick',
                            version = sj.constant.QuoteVersion.v1
                            )

    while True:

        time.sleep(1)

        current_time = datetime.datetime.now()

        if current_time.second == 0 :

            kbars[code] = ticks_to_kbars(ticks[code], interval='1Min')
            kbars[code] = kbars[code][prev_trading_date:]

            str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')

            try:
                current_kbars = kbars[code].loc[str_current_time]
            except:
                continue

            print(current_kbars)


In [83]:
tbot('2330')

--- 啟動 Tbot #2330 ---
Response Code: 200 | Event Code: 16 | Info: Subscription Already Exists - Topic 'TIC/v1/STK/*/TSE/2330' Subscribe: Response code: '400'; Flags: 'c';  | Event: Subscribe or Unsubscribe ok


KeyboardInterrupt: 